Install Python libraries in a new notebook `.ipynb`.

* In this method, we are using a [notebook-scoped Python library](https://learn.microsoft.com/en-us/azure/databricks/libraries/notebooks-python-libraries) which does not persist between sessions.
* A cluster library is recommended if you're sharing a set of classic compute across the team.

In [ ]:
%pip install geopandas rasterio
%restart_python